# Cross Validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Advertising.csv')

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


<h2> 1. Train | Test split procedure </h2>   

<ol start='0'>  
 <li>Clean and adjust the data as necessary for X and y </li>  
 <li>Split data in Train and Test for both X and y</li>  
 <li>Fit/Train Scaler on training X data</li>  
 <li>Scale  X test data</li>
 <li>Create Model</li>
 <li>Fit/Train model on X train data</li>
 <li>Evaluate model on X test data (by creating test_predictions and compareto y_test)</li>
 <li>Adjust parameters as Necessary and repeat steps 5 and 6</li>
     
</ol> 

In [4]:
#Create X
X = df.drop('sales',axis=1)

In [5]:
#create y
y = df['sales']

In [6]:
# Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [7]:
#Scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()    #create an instance

scaler.fit(X_train)   # .fit() calculates the mean and standard dev. Here it is using only train_data
                      # .fit() we use only train data and not test_data to avoid leakage

#scaled X_train, scaled X_test
X_train = scaler.transform(X_train)  #transfroms X_train to get scaled data where mean and std dev calculated from X_train
X_test = scaler.transform(X_test)    #transfroms X_test to get scaled data where mean and std dev calculated from X_train
                                     # 'scaler' is defined only on X_train and not X_test, to avoid data leakage.


In [8]:
# Create Model
from sklearn.linear_model import Ridge

In [9]:
# model one choosing alpha = 100
model = Ridge(alpha=100)

In [10]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [11]:
y_pred = model.predict(X_test)

In [12]:
# Evaluation
from sklearn.metrics import mean_squared_error

In [13]:
mean_squared_error(y_test, y_pred)

7.34177578903413

In [14]:
# Changing alpha and Re-evaluate
model_2 = Ridge(alpha=1)

In [15]:
model_2.fit(X_train, y_train)

Ridge(alpha=1)

In [16]:
y_pred_1 = model_2.predict(X_test)

In [17]:
mean_squared_error(y_test,y_pred_1)

2.319021579428752

We see the error is reduced by changing alpha parameter. So how to get the best possible alpha??    
we can do for loop for changing alpha (brute force) - not recommended and no guarantee of getting good alpha.  
We are using test data to go back and adjust parameters (like alpha) to get better value - $\implies$ there is some leakage or not completely unseen data.   

Let's look other validation methods.

--------------

<h2> 2. Train | Validation | Test </h2>
<ol start='0'>  
 <li>Clean and adjust the data as necessary for X and y </li>  
 <li>Split data in Train, Validation and Test for both X and y</li>  
 <li>Fit/Train Scaler on training X data</li>  
 <li>Scale  X validation data</li>
 <li>Create Model</li>
 <li>Fit/Train model on X train data</li>
 <li>Evaluate model on X validation data (by creating test_predictions and compare to y_validation)</li>
 <li>Adjust parameters as Necessary and repeat steps 5 and 6</li>
 <li>Final metrics do it on Test Data (can't go back after this)</li>
     
</ol>

In [18]:
# Create X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [19]:
# Split into three parts (70% | 15% | 15%) for (Train | Validation | Test)
from sklearn.model_selection import train_test_split

#Step 1- split 70% as train data and 30% as other data
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)
    
#Step 2 - Split the other data in 50% as validation and 50% as test 
    #(it makes validation as 15% and test as 15% of whole data X, y)
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=101)


In [20]:
print('Whole data length: ', len(df))
print('X_train data length: ',len(X_train), ', percent',100*len(X_train)/len(df),'%')
print('X_val data length: ', len(X_val), ', percent',100*len(X_val)/len(df),'%')
print('X_test data length: ', len(X_test), ', percent',100*len(X_test)/len(df),'%')

Whole data length:  200
X_train data length:  140 , percent 70.0 %
X_val data length:  30 , percent 15.0 %
X_test data length:  30 , percent 15.0 %


In [21]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [22]:
# Create Model
from sklearn.linear_model import Ridge

#model_1
model_1 = Ridge(alpha=100)

In [23]:
model_1.fit(X_train, y_train)

Ridge(alpha=100)

In [24]:
y_val_pred = model_1.predict(X_val)

In [25]:
#Evaluation
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val,y_val_pred)

7.320101458823871

In [26]:
#do for model_2
model_2 = Ridge(alpha=1)

model_2.fit(X_train,  y_train)
y_val_pred_2 = model_2.predict(X_val)

In [27]:
#Evaluation
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val,y_val_pred_2)

2.383783075056986

In [28]:
# If you feel its ok (error is acceptable) after evaluation - Go to test data
#predictions using X_test
y_final_testpred = model_2.predict(X_test)

#Evaluate
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_final_testpred)

2.254260083800517

-------------------

<h2> 3. cross_val_score </h2>
<ol start='0'>  
 <li>Clean and adjust the data as necessary for X and y </li>  
 <li>Split data in Train and Test for both X and y</li>  
 <li>Fit/Train Scaler on training X data</li>  
 <li>Scale  X test data</li>
 <li>Create Model</li>
 <li>Use cross_va_score and set the parameters</li>
 <li>See the average of the scores</li>
 <li>Adjust parameters as Necessary and repeat steps 5 and 6</li>
 <li>Look for best accepted score</li>
 <li>Final metrics, use the model on Test Data (can't go back after this)</li>
     
</ol>

https://scikit-learn.org/stable/modules/model_evaluation.html

In [29]:
# Create X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [30]:
# Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [31]:
#Scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()    
scaler.fit(X_train) 

#scaled X_train, scaled X_test
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
# Create Model
from sklearn.linear_model import Ridge
model = Ridge(alpha=100)

In [33]:
# use cross_val_score
from sklearn.model_selection import cross_val_score

In [34]:
#Use cross_va_score and set the parameters
#https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,scoring='neg_mean_squared_error',cv=5)

In [35]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [36]:
#See the average of the scores
abs(scores.mean())

8.215396464543607

In [37]:
#Changing the model parameter
model_one = Ridge(alpha=1)

In [38]:
scores = cross_val_score(model_one,X_train,y_train,scoring='neg_mean_squared_error',cv=5) #cross_val_score fits the model
scores

array([-3.15513238, -1.58086982, -5.40455562, -2.21654481, -4.36709384])

In [39]:
abs(scores.mean())

3.344839296530696

In [40]:
#Final metrics, use the model on Test Data (can't go back after this)

#Need to fit the model first
model_one.fit(X_train,y_train)

Ridge(alpha=1)

In [41]:
y_final_test_pred = model_one.predict(X_test)

In [42]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

<h1> Cross Validation with cross_validate </h1>

https://scikit-learn.org/stable/modules/model_evaluation.html

In [43]:
#Create X and y
X = df.drop('sales',axis=1)
y = df['sales']

#Train and Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

#Scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
from sklearn.model_selection import cross_validate

In [45]:
model = Ridge(alpha=100)

In [46]:
scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error','neg_mean_absolute_error'], cv=10)

In [47]:
scores   #looks clumsy convert to dataframe

{'fit_time': array([0.00199914, 0.00099683, 0.00199509, 0.00200081, 0.00199914,
        0.00224018, 0.00100708, 0.00206923, 0.0010016 , 0.00108361]),
 'score_time': array([0.00221729, 0.0010004 , 0.00199986, 0.00099921, 0.00112557,
        0.00100374, 0.00094104, 0.00102496, 0.00101972, 0.00113893]),
 'test_neg_mean_squared_error': array([ -6.06067062, -10.62703078,  -3.99342608,  -5.00949402,
         -9.14179955, -13.08625636,  -3.83940454,  -9.05878567,
         -9.05545685,  -5.77888211]),
 'test_neg_mean_absolute_error': array([-1.8102116 , -2.54195751, -1.46959386, -1.86276886, -2.52069737,
        -2.45999491, -1.45197069, -2.37739501, -2.44334397, -1.89979708])}

In [48]:
scores = pd.DataFrame(scores)
scores

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.001999,0.002217,-6.060671,-1.810212
1,0.000997,0.001000,-10.627031,-2.541958
2,0.001995,0.002000,-3.993426,-1.469594
3,0.002001,0.000999,-5.009494,-1.862769
4,0.001999,0.001126,-9.141800,-2.520697
5,0.002240,0.001004,-13.086256,-2.459995
6,0.001007,0.000941,-3.839405,-1.451971
7,0.002069,0.001025,-9.058786,-2.377395
8,0.001002,0.001020,-9.055457,-2.443344
9,0.001084,0.001139,-5.778882,-1.899797


In [49]:
scores.mean()

fit_time                        0.001639
score_time                      0.001247
test_neg_mean_squared_error    -7.565121
test_neg_mean_absolute_error   -2.083773
dtype: float64

In [50]:
# improving model
model = Ridge(alpha=1)

In [51]:
scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error','neg_mean_absolute_error'], cv=10)

In [52]:
scores = pd.DataFrame(scores)
scores

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.002010,0.001986,-2.962508,-1.457174
1,0.003252,0.001002,-3.057378,-1.555308
2,0.001999,0.001000,-2.173740,-1.238770
3,0.002001,0.000999,-0.833034,-0.768938
4,0.000999,0.001581,-3.464018,-1.434489
5,0.001998,0.000999,-8.232647,-1.494316
6,0.000999,0.001002,-1.905864,-1.081362
7,0.003002,0.002064,-2.765048,-1.250011
8,0.002931,0.001987,-4.989505,-1.580971
9,0.001007,0.000991,-2.846438,-1.223326


In [53]:
scores.mean()   #error has decreased

fit_time                        0.002020
score_time                      0.001361
test_neg_mean_squared_error    -3.323018
test_neg_mean_absolute_error   -1.308467
dtype: float64

In [54]:
# Use this 2nd tried model and proceed on X_test
# First fit the model on train
model.fit(X_train,y_train)

Ridge(alpha=1)

In [55]:
y_final_pred = model.predict(X_test)

In [56]:
mean_squared_error(y_test, y_final_pred)

2.319021579428752

-----

-------

<h1> Grid Search </h1>

We can use grid search for using variety of combinations of hyperparameters 

In [57]:
# Create X and y
X = df.drop('sales',axis=1)
y = df['sales']

#Split train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Scale daat
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# Model
from sklearn.linear_model import ElasticNet

In [59]:
#help(ElasticNet)

A search consists of:

* an estimator (regressor or classifier such as sklearn.svm.SVC());
* a parameter space;
* a method for searching or sampling candidates;
* a cross-validation scheme 
* a score function.

In [60]:
base_elastic_model = ElasticNet()

In [61]:
param_grid = {'alpha':[0.1,1,5,10,50,100], 'l1_ratio':[.1,.5,.7,.9,.95,.99,1]}

In [62]:
from sklearn.model_selection import GridSearchCV

In [63]:
grid_model = GridSearchCV(estimator=base_elastic_model, param_grid=param_grid,
                         scoring='neg_mean_squared_error', cv=5, verbose=2)   #verbose tells whats happening, 
                                                                            #by giving integer we limit not to say every step

In [64]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.7; total time=   0.0s
[CV] END ............................alpha=0.1,

[CV] END ...........................alpha=100, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=100, l1_ratio=0.99; total time=   0.0s
[CV] END ...........................alpha=100, l1_ratio=0.99; total time=   0.0s
[CV] END ..............................alpha=100, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=100, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=100, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=100, l1_ratio=1; total time=   0.0s
[CV] END ..............................alpha=100, l1_ratio=1; total time=   0.0s


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=2)

In [65]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

In [66]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [67]:
pd.DataFrame(grid_model.cv_results_).head()  #we can see here the error for each combination

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002950,0.001788,0.000802,0.000401,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-3.453021,-1.405190,-5.789125,-2.187302,-4.645576,-3.496043,1.591601,7
1,0.001410,0.000479,0.000823,0.000413,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-3.325440,-1.427522,-5.595610,-2.163089,-4.451679,-3.392668,1.506827,6
2,0.001796,0.000400,0.000820,0.000412,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-3.269880,-1.442432,-5.502437,-2.163950,-4.356738,-3.347088,1.462765,5
3,0.001457,0.000450,0.000806,0.000403,0.1,0.9,"{'alpha': 0.1, 'l1_ratio': 0.9}",-3.221397,-1.465339,-5.416447,-2.173493,-4.263887,-3.308112,1.417693,4
4,0.001388,0.000472,0.000201,0.000401,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.213052,-1.472417,-5.396258,-2.177452,-4.241080,-3.300052,1.406248,3


In [68]:
#using best model to fit
y_pred = grid_model.predict(X_test)  #it automatically takes best model

In [69]:
from sklearn.metrics import mean_squared_error

In [70]:
mean_squared_error(y_test,y_pred)

2.3873426420874737

------